In [ ]:
import pathlib
import os
if 'TSL_SCHOOL_DIR' in os.environ:
     if any( (p/".git").is_dir() for p in
(pathlib.Path(".").absolute().resolve()/"dummy").parents ):
         raise RuntimeError('Please copy notebook to a work directory')

# Geometry optimization for Aluminum      



In the first exercise we have seen how to run DFT calculations for a material that present a band gap, here we see here how to run them for a metal. To save time, in this example we skip for the convergence tests but convergence tests are mandatory for metals like they are insulators. 

While in insulators the occupation is fixed and you have a definite separation between occupied and unoccupied electronic bands, in metals we have to deal with partially occupied bands in which the energy level that separates occupied and unoccupied states (the Fermi level) falls within the band.

This requires the use of smearing techniques to stabilize the convergence of the self consistent cycle. Due to numerical errors in fact, the electrons may occupy the unoccupied states during some iterations, making the algorithm unstable. With the smearing techniques the problem is solved by replacing the fixed occupation number (either 0 or 1) with a smoothly varying function to partially occupy the states around the Fermi level. 

As for the cutoff also the smearing parameter needs to be converged, usually this convergence is done in conjunction with the convergence tests for the k-points. The two quantities are in fact linked since a larger number of k-points means less numerical noise and thus require a smaller smearing to converge and vice-versa.

As in our first example let's start getting the coordinates and completing the pw.x input file. First we run 

                    cif2cell  1502689.cif --print-cartesian >> coordinates.dat
                    
and we replace the atomic positions and cell vectors in the input file

In [1]:
cat ./coordinates.dat

CIF2CELL 2.0.0
2023-05-12 18:13
Output for None (?)

 BIBLIOGRAPHIC INFORMATION
Mulder, Fokko M. et al., The Journal of Physical Chemistry C 114, 10648- (2010) 
 INPUT CELL INFORMATION
Symmetry information:
Cubic crystal system.
Space group number     : 225
Hall symbol            : -F 4 2 3
Hermann-Mauguin symbol : Fm-3m

Lattice parameters:
          a           b           c 
  4.0339000   4.0339000   4.0339000 
      alpha        beta       gamma 
 90.0000000  90.0000000  90.0000000 
Representative sites :
Atom            x           y           z 
Al      0.0000000   0.0000000   0.0000000

 OUTPUT CELL INFORMATION
Symmetry information:
Cubic crystal system.
Space group number     : 225
Hall symbol            : -F 4 2 3
Hermann-Mauguin symbol : Fm-3m

Bravais lattice vectors :
  2.0169500   2.0169500   0.0000000 
  2.0169500   0.0000000   2.0169500 
  0.0000000   2.0169500   2.0169500 
All sites, (cartesian coordinates):
Atom            x           y           z 
Al      0.0000000  

In [3]:
cat ./pw.vcrelax.aluminum.in

&CONTROL
  calculation = 'vc-relax'
  etot_conv_thr =   1.0000000000d-05
  forc_conv_thr =   1.0000000000d-04
  outdir = './'
  prefix = 'al'
  pseudo_dir = './pseudo/'
  tprnfor = .true.
  tstress = .true.
  verbosity = 'high'
/
&SYSTEM
  ecutrho =   2.4000000000d+02
  ecutwfc =   3.0000000000d+01
  ibrav = 0
  nat = 1
  ntyp = 1
  
  ! you need to specify that the occupation is not fixed like in insulators with a gap
  occupations = 'smearing'
  
  ! type of smearing that you want to use
  smearing = 'cold'
  
  ! value of the smearing
  degauss =   0.02
/
&ELECTRONS
  conv_thr =   2.0000000000d-10
  electron_maxstep = 80
  mixing_beta =   4.0000000000d-01
/
&IONS
/
&CELL
/
ATOMIC_SPECIES
Al     26.981538 Al.pbe-n-kjpaw_psl.1.0.0.UPF
K_POINTS automatic
! k-points need to be denser for metals than for insulators
14 14 14 0 0 0
ATOMIC_POSITIONS angstrom
Al           0.0000000000       0.0000000000       0.0000000000 
CELL_PARAMETERS angstrom
      2.0200000000       2.0200000000       

Again, before running the simulation we can verify that the structure is correct using Xcrysden:
     
             xcrysden --pwi pw.vcrelax.aluminum.in

If everything is correct we can run the simulation using pw.x

             pw.x -inp pw.vcrelax.aluminum.in >> pw.vcrelax.aluminum.out

We can see how the Fermi level is recomputed at each iteration

In [4]:
! grep "Fermi" ./pw.vcrelax.aluminum.out

     the Fermi energy is     7.9032 ev
     the Fermi energy is     7.9126 ev
     the Fermi energy is     7.9267 ev
     the Fermi energy is     7.9267 ev
